In [103]:
import numpy as np
import pandas as pd

In [104]:
data = pd.read_csv('/Users/prateekanand/Desktop/CS148/Travel-Purpose-Prediction/csv/trippub.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(data.head())

    HOUSEID  PERSONID  TDTRPNUM  STRTTIME  ENDTIME  TRVLCMIN  TRPMILES  TRPTRANS  TRPACCMP  TRPHHACC  VEHID  TRWAITTM  NUMTRANS  TRACCTM  DROP_PRK  TREGRTM  WHODROVE  WHYFROM  LOOP_TRIP  TRPHHVEH  HHMEMDRV  HH_ONTD  NONHHCNT  NUMONTRP  PSGR_FLG  PUBTRANS TRIPPURP  DWELTIME  TDWKND  VMT_MILE  DRVR_FLG  WHYTRP1S  ONTD_P1  ONTD_P2  ONTD_P3  ONTD_P4  ONTD_P5  ONTD_P6  ONTD_P7  ONTD_P8  ONTD_P9  ONTD_P10  ONTD_P11  ONTD_P12  ONTD_P13      TDCASEID  TRACC_WLK  TRACC_POV  TRACC_BUS  TRACC_CRL  TRACC_SUB  TRACC_OTH  TREGR_WLK  TREGR_POV  TREGR_BUS  TREGR_CRL  TREGR_SUB  TREGR_OTH  WHYTO  TRAVDAY  HOMEOWN  HHSIZE  HHVEHCNT  HHFAMINC  DRVRCNT HHSTATE  HHSTFIPS  NUMADLT  WRKCOUNT  TDAYDATE  HHRESP  LIF_CYC  MSACAT  MSASIZE  RAIL  URBAN  URBANSIZE  URBRUR  GASPRICE  CENSUS_D  CENSUS_R  CDIVMSAR  HH_RACE  HH_HISP HH_CBSA  SMPLSRCE  R_AGE  EDUC  R_SEX  PRMACT  PROXY  WORKER  DRIVER      WTTRDFIN  WHYTRP90   TRPMILAD  R_AGE_IMP  R_SEX_IMP  VEHTYPE OBHUR DBHUR  OTHTNRNT  OTPPOPDN  OTRESDN  OTEEMPDN  O

Preprocessing

In [105]:
# Drop columns related to trip purpose, also drop HH_CBSA because most values are XXXXX and cannot impute those
drop = ['WHYTO', 'WHYFROM', 'TRIPPURP', 'WHYTRP1S', 'WHYTRP90', 'HH_CBSA']
y = data['WHYTRP1S']
X = data.drop(columns=drop, axis=1)

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print(categorical_cols)

categorical_idx = [X.columns.get_loc(col) for col in categorical_cols]
print(categorical_idx)

print(X.head())

['HHSTATE', 'OBHUR', 'DBHUR']
[61, 93, 94]
    HOUSEID  PERSONID  TDTRPNUM  STRTTIME  ENDTIME  TRVLCMIN  TRPMILES  TRPTRANS  TRPACCMP  TRPHHACC  VEHID  TRWAITTM  NUMTRANS  TRACCTM  DROP_PRK  TREGRTM  WHODROVE  LOOP_TRIP  TRPHHVEH  HHMEMDRV  HH_ONTD  NONHHCNT  NUMONTRP  PSGR_FLG  PUBTRANS  DWELTIME  TDWKND  VMT_MILE  DRVR_FLG  ONTD_P1  ONTD_P2  ONTD_P3  ONTD_P4  ONTD_P5  ONTD_P6  ONTD_P7  ONTD_P8  ONTD_P9  ONTD_P10  ONTD_P11  ONTD_P12  ONTD_P13      TDCASEID  TRACC_WLK  TRACC_POV  TRACC_BUS  TRACC_CRL  TRACC_SUB  TRACC_OTH  TREGR_WLK  TREGR_POV  TREGR_BUS  TREGR_CRL  TREGR_SUB  TREGR_OTH  TRAVDAY  HOMEOWN  HHSIZE  HHVEHCNT  HHFAMINC  DRVRCNT HHSTATE  HHSTFIPS  NUMADLT  WRKCOUNT  TDAYDATE  HHRESP  LIF_CYC  MSACAT  MSASIZE  RAIL  URBAN  URBANSIZE  URBRUR  GASPRICE  CENSUS_D  CENSUS_R  CDIVMSAR  HH_RACE  HH_HISP  SMPLSRCE  R_AGE  EDUC  R_SEX  PRMACT  PROXY  WORKER  DRIVER      WTTRDFIN   TRPMILAD  R_AGE_IMP  R_SEX_IMP  VEHTYPE OBHUR DBHUR  OTHTNRNT  OTPPOPDN  OTRESDN  OTEEMPDN  OBHTNRNT  O

In [106]:
# Invalid/missing values are reported as negative values in the dataframe or the string '-9'
X = X.map(lambda x: np.nan if ((isinstance(x, (int, float)) and x < 0) or (isinstance(x, str) and x == '-9')) else x)

# Replace these two columns with mode since they are categorical
mode = X['OBHUR'].mode()
X['OBHUR'].fillna(mode, inplace=True)
mode = X['DBHUR'].mode()
X['DBHUR'].fillna(mode, inplace=True)

print(X.head())

    HOUSEID  PERSONID  TDTRPNUM  STRTTIME  ENDTIME  TRVLCMIN  TRPMILES  TRPTRANS  TRPACCMP  TRPHHACC  VEHID  TRWAITTM  NUMTRANS  TRACCTM  DROP_PRK  TREGRTM  WHODROVE  LOOP_TRIP  TRPHHVEH  HHMEMDRV  HH_ONTD  NONHHCNT  NUMONTRP  PSGR_FLG  PUBTRANS  DWELTIME  TDWKND  VMT_MILE  DRVR_FLG  ONTD_P1  ONTD_P2  ONTD_P3  ONTD_P4  ONTD_P5  ONTD_P6  ONTD_P7  ONTD_P8  ONTD_P9  ONTD_P10  ONTD_P11  ONTD_P12  ONTD_P13      TDCASEID  TRACC_WLK  TRACC_POV  TRACC_BUS  TRACC_CRL  TRACC_SUB  TRACC_OTH  TREGR_WLK  TREGR_POV  TREGR_BUS  TREGR_CRL  TREGR_SUB  TREGR_OTH  TRAVDAY  HOMEOWN  HHSIZE  HHVEHCNT  HHFAMINC  DRVRCNT HHSTATE  HHSTFIPS  NUMADLT  WRKCOUNT  TDAYDATE  HHRESP  LIF_CYC  MSACAT  MSASIZE  RAIL  URBAN  URBANSIZE  URBRUR  GASPRICE  CENSUS_D  CENSUS_R  CDIVMSAR  HH_RACE  HH_HISP  SMPLSRCE  R_AGE  EDUC  R_SEX  PRMACT  PROXY  WORKER  DRIVER      WTTRDFIN   TRPMILAD  R_AGE_IMP  R_SEX_IMP  VEHTYPE OBHUR DBHUR  OTHTNRNT  OTPPOPDN  OTRESDN  OTEEMPDN  OBHTNRNT  OBPPOPDN  OBRESDN  DTHTNRNT  DTPPOPDN  DTRES

In [107]:
# Then impute the numerical nan values with mean
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()

for col in numerical_cols:
    mean = X[col].mean()
    X[col].fillna(mean, inplace=True)

print(X.head())

    HOUSEID  PERSONID  TDTRPNUM  STRTTIME  ENDTIME  TRVLCMIN  TRPMILES  TRPTRANS  TRPACCMP  TRPHHACC  VEHID   TRWAITTM  NUMTRANS   TRACCTM  DROP_PRK    TREGRTM  WHODROVE  LOOP_TRIP  TRPHHVEH  HHMEMDRV  HH_ONTD  NONHHCNT  NUMONTRP  PSGR_FLG  PUBTRANS    DWELTIME  TDWKND  VMT_MILE  DRVR_FLG  ONTD_P1  ONTD_P2  ONTD_P3   ONTD_P4   ONTD_P5   ONTD_P6   ONTD_P7   ONTD_P8   ONTD_P9  ONTD_P10  ONTD_P11  ONTD_P12  ONTD_P13      TDCASEID  TRACC_WLK  TRACC_POV  TRACC_BUS  TRACC_CRL  TRACC_SUB  TRACC_OTH  TREGR_WLK  TREGR_POV  TREGR_BUS  TREGR_CRL  TREGR_SUB  TREGR_OTH  TRAVDAY  HOMEOWN  HHSIZE  HHVEHCNT  HHFAMINC  DRVRCNT HHSTATE  HHSTFIPS  NUMADLT  WRKCOUNT  TDAYDATE  HHRESP  LIF_CYC  MSACAT  MSASIZE  RAIL  URBAN  URBANSIZE  URBRUR  GASPRICE  CENSUS_D  CENSUS_R  CDIVMSAR  HH_RACE  HH_HISP  SMPLSRCE  R_AGE  EDUC  R_SEX  PRMACT  PROXY  WORKER  DRIVER      WTTRDFIN   TRPMILAD  R_AGE_IMP  R_SEX_IMP  VEHTYPE OBHUR DBHUR  OTHTNRNT  OTPPOPDN  OTRESDN  OTEEMPDN  OBHTNRNT  OBPPOPDN  OBRESDN  DTHTNRNT  DTP

In [108]:
# One-hot encode categorical features
X = pd.get_dummies(X, columns = categorical_cols)
X.replace({True: 1, False: 0}, inplace=True)

print(X.shape)
print(X.head())

(923572, 167)
    HOUSEID  PERSONID  TDTRPNUM  STRTTIME  ENDTIME  TRVLCMIN  TRPMILES  TRPTRANS  TRPACCMP  TRPHHACC  VEHID   TRWAITTM  NUMTRANS   TRACCTM  DROP_PRK    TREGRTM  WHODROVE  LOOP_TRIP  TRPHHVEH  HHMEMDRV  HH_ONTD  NONHHCNT  NUMONTRP  PSGR_FLG  PUBTRANS    DWELTIME  TDWKND  VMT_MILE  DRVR_FLG  ONTD_P1  ONTD_P2  ONTD_P3   ONTD_P4   ONTD_P5   ONTD_P6   ONTD_P7   ONTD_P8   ONTD_P9  ONTD_P10  ONTD_P11  ONTD_P12  ONTD_P13      TDCASEID  TRACC_WLK  TRACC_POV  TRACC_BUS  TRACC_CRL  TRACC_SUB  TRACC_OTH  TREGR_WLK  TREGR_POV  TREGR_BUS  TREGR_CRL  TREGR_SUB  TREGR_OTH  TRAVDAY  HOMEOWN  HHSIZE  HHVEHCNT  HHFAMINC  DRVRCNT  HHSTFIPS  NUMADLT  WRKCOUNT  TDAYDATE  HHRESP  LIF_CYC  MSACAT  MSASIZE  RAIL  URBAN  URBANSIZE  URBRUR  GASPRICE  CENSUS_D  CENSUS_R  CDIVMSAR  HH_RACE  HH_HISP  SMPLSRCE  R_AGE  EDUC  R_SEX  PRMACT  PROXY  WORKER  DRIVER      WTTRDFIN   TRPMILAD  R_AGE_IMP  R_SEX_IMP  VEHTYPE  OTHTNRNT  OTPPOPDN  OTRESDN  OTEEMPDN  OBHTNRNT  OBPPOPDN  OBRESDN  DTHTNRNT  DTPPOPDN 

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

0         20
1          1
2          1
3         10
4         20
          ..
923567    10
923568    50
923569    10
923570    40
923571     1
Name: WHYTRP1S, Length: 923572, dtype: int64
